# Measurement Grouping

Since current quantum hardware is limited to single-qubit projective measurement, only terms commuting within individual qubit's subspace can be measured together. These terms are said to be qubit-wise commuting (QWC). 

Thus, one can not measure the entire electronic Hamiltonian $\hat H$ at once, and instead needs to separate it into fragments. 
$$\hat H = \sum_n \hat H_n$$
where each $\hat H_n$ is a QWC fragment. 

In [1]:
from utility import * 
import time

Here we use $H_2$ as an example for finding QWC fragments. Notice below that each fragment has the same terms on all qubits.  

To show differences between QWC and more advanced grouping, we didn't use the qubit-tappering techinique shown in step 2.

In [2]:
h2 = get_qubit_hamiltonian(mol='h2', geometry=1, basis='sto3g', qubit_transf='jw')

qwc_list = get_qwc_group(h2)
print('Fragments 1: \n{}\n'.format(qwc_list[4]))
print('Fragments 2:\n{}\n'.format(qwc_list[1]))
print('Number of fragments: {}'.format(len(qwc_list)))

Fragments 1: 
0.13716572937099503 [Z0] +
0.15660062488237958 [Z0 Z1] +
0.10622904490856082 [Z0 Z2] +
0.15542669077992838 [Z0 Z3] +
0.13716572937099503 [Z1] +
0.15542669077992838 [Z1 Z2] +
0.10622904490856082 [Z1 Z3] +
-0.13036292057109122 [Z2] +
0.1632676867356435 [Z2 Z3] +
-0.13036292057109122 [Z3]

Fragments 2:
-0.049197645871367574 [X0 X1 Y2 Y3]

Number of fragments: 5


By applying extra unitaries, one may rotate more terms of $\hat H$ into a QWC fragment.  

Recall that in digital quantum computing, the expectation value of $\hat H_n$ given a trial wavefunction $|\psi\rangle$ is 
$$ E_n =\ \langle\psi| \hat H_n | \psi\rangle$$
Inserting unitary transformation $\hat U_n$ does not change the expectation value.
$$ E_n =\ \langle\psi| \hat U_n^\dagger \hat U_n \hat H_n \hat U_n^\dagger \hat U_n  |\psi\rangle$$ 
This nonetheless changes the trial wavefunction and the terms to be measured. 
$$ |\psi\rangle \rightarrow \hat U_n |\psi\rangle = |\phi\rangle$$
$$ \hat H_n \rightarrow \hat U_n \hat H_n \hat U_n^\dagger = \hat A_n$$
The transformation of $|\psi \rangle$ can be done on the quantum computer, and the transformation of $\hat H_n$ is possible on the classical computer. 

Now, although $\hat A_n$ needs to be a QWC fragment to be measurable on a quantum computer, $\hat H_n$ does not. 
Instead, if we restrict $\hat U_n$ to be a clifford operation, the terms in $\hat H$ need only mutually commute. 

Here, we obtain measurable parts of $H_2$ by partitioning its terms into mutually commuting fragments. 

In [3]:
comm_groups = get_commuting_group(h2)
print('Number of mutually commuting fragments: {}'.format(len(comm_groups)))
print('The first commuting group')
print(comm_groups[1])

Number of mutually commuting fragments: 2
The first commuting group
-0.32760818967480915 [] +
-0.049197645871367574 [X0 X1 Y2 Y3] +
0.049197645871367574 [X0 Y1 Y2 X3] +
0.049197645871367574 [Y0 X1 X2 Y3] +
-0.049197645871367574 [Y0 Y1 X2 X3] +
0.15660062488237958 [Z0 Z1] +
0.10622904490856082 [Z0 Z2] +
0.15542669077992838 [Z0 Z3] +
0.15542669077992838 [Z1 Z2] +
0.10622904490856082 [Z1 Z3] +
0.1632676867356435 [Z2 Z3]


To see this fragment is indeed measurable, one can construct the corresponding unitary operator $\hat U_n$.

In [4]:
uqwc = get_qwc_unitary(comm_groups[1])
print('This is unitary, U * U^+ = I ')
print(uqwc * uqwc)

This is unitary, U * U^+ = I 
(0.9999999999999996+0j) []


Applying this unitary gives the qubit-wise commuting form of the first mutually commuting group

In [5]:
qwc = remove_complex(uqwc * comm_groups[1] * uqwc)
print(qwc)

-0.3276081896748089 [] +
0.15542669077992824 [X0] +
0.15660062488237952 [X0 X1] +
0.049197645871367546 [X0 X1 Z3] +
0.10622904490856076 [X0 X2] +
-0.049197645871367546 [X0 Z3] +
0.10622904490856076 [X1] +
0.15542669077992824 [X1 X2] +
-0.049197645871367546 [X1 X2 Z3] +
0.16326768673564335 [X2] +
0.049197645871367546 [X2 Z3]


In addition, current quantum computer can measure only the $z$ operators. Thus, QWC fragments with $x$ or $y$ operators require extra single-qubit unitaries that rotate them into $z$.  

In [6]:
uz = get_zform_unitary(qwc)
print("Checking whether U * U^+ is identity: {}".format(uz * uz))

allz = remove_complex(uz * qwc * uz)
print("\nThe all-z form of qwc fragment:\n{}".format(allz))

Checking whether U * U^+ is identity: 0.9999999999999998 []

The all-z form of qwc fragment:
-0.3276081896748088 [] +
0.15542669077992818 [Z0] +
0.15660062488237944 [Z0 Z1] +
0.04919764587136753 [Z0 Z1 Z3] +
0.1062290449085607 [Z0 Z2] +
-0.04919764587136753 [Z0 Z3] +
0.1062290449085607 [Z1] +
0.15542669077992818 [Z1 Z2] +
-0.04919764587136753 [Z1 Z2 Z3] +
0.16326768673564326 [Z2] +
0.04919764587136753 [Z2 Z3]


In [ ]:
molecules=['h2','h4','lih','h2o','n2','nh3']
methods=['hf','cisd','ccsd','fci']
basies=['sto-3g','6-31g']
qubit_transfms=['jw','bk'] 

qubit_transf = 'jw' # Jordan-Wigner transformations

for mol in molecules:

    for basis in basies:

        for qubit_transf in qubit_transfms:
                
            start=time.time()
            
            H = get_qubit_hamiltonian(mol, geometry=1, basis=basis, qubit_transf=qubit_transf)

            qwc_list = get_qwc_group(H)
            print('\nmolecule:',mol.upper(),' in ',basis.upper(),'basis using',qubit_transf.upper(),
                  'q-bits map; number of fragments: {}'.format(len(qwc_list)))

            comm_groups = get_commuting_group(H)
            print('Number of mutually commuting fragments: {}'.format(len(comm_groups)))
            
            if len(comm_groups) < 10:

                for i,comm_group in enumerate(comm_groups):

                    uqwc = get_qwc_unitary(comm_groups[i+1])

                    if np.random.random() <0.2:
                        print("\tRandom test of the Unitary...")
                        tmp=remove_complex(uqwc * uqwc)
                        if np.abs(1-tmp.terms[()])>10**(-6):
                            print('Unitary problem: I=/=U * U^+ is:',tmp)
                    else:
                        test=False

                    qwc = remove_complex(uqwc * comm_groups[i+1] * uqwc)
                    uz = get_zform_unitary(qwc)

                    if np.random.random() <0.2:
                        print("\tRandom test of the UZ Unitary...")
                        tmp=remove_complex(uz * uz)
                        if abs(1-tmp.terms[()])>10**(-6) and test:
                            print('UZ unitary problem: I=/=U * U^+ is:',tmp)

                    allz = remove_complex(uz * qwc * uz)
                    print("\tqwc fragment {} has {} all-z elements (original elemets {}).".format(
                        i+1,len(allz.terms),len(comm_groups[i+1].terms)))
                      
            print("\tComputainal time:",time.time()-start)          


molecule: H2  in  STO-3G basis using JW q-bits map; number of fragments: 5
Number of mutually commuting fragments: 2
	qwc fragment 1 has 11 all-z elements (original elemets 11).
	qwc fragment 2 has 4 all-z elements (original elemets 4).
	Computainal time: 1.308319330215454

molecule: H2  in  STO-3G basis using BK q-bits map; number of fragments: 3
Number of mutually commuting fragments: 2
	qwc fragment 1 has 11 all-z elements (original elemets 11).
	Random test of the Unitary...
	qwc fragment 2 has 4 all-z elements (original elemets 4).
	Computainal time: 0.21230554580688477

molecule: H2  in  6-31G basis using JW q-bits map; number of fragments: 68
Number of mutually commuting fragments: 9
	Random test of the UZ Unitary...
	qwc fragment 1 has 29 all-z elements (original elemets 29).
	qwc fragment 2 has 24 all-z elements (original elemets 24).
	qwc fragment 3 has 20 all-z elements (original elemets 20).
	Random test of the UZ Unitary...
	qwc fragment 4 has 20 all-z elements (original 